In [ ]:
import os
import yaml
import requests
import json

def _get_config(service_str):
    path = os.path.join(os.path.dirname(os.path.realpath('__file__')),'config/config.txt')
    with open(path, 'r') as config_file:
        config = yaml.load(config_file)
    return config[service_str]

#https://qpx-express-demo.itasoftware.com/

def get_flight_results(payload):
    config = _get_config('qpx_express')
    url = '{}?key={}'.format(config['url'],config['project_key'])
    r = requests.post(url, 
                      data = json.dumps(payload), 
                      headers = config['headers'])
    
    return json.loads(r.text)

In [ ]:
json_request = {
    "request": {
        "passengers": {
            "adultCount": 1
        },
        "slice": [
            {
                "origin": "BOS",
                "destination": "LAX",
                "date": "2016-12-30",
                "maxStops": 0
            }
        ],
        "solutions": 20,
        "refundable": False
    }
}

get_flight_results(json_request)